In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('bmh')

In [2]:
from tqdm import tqdm_notebook

from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import r2_score
from sklearn.externals import joblib

from sorder.models.linear_regression import Corpus
from sorder.metrics import Metrics

In [3]:
dv = joblib.load('dv.bin')

In [4]:
vocab = joblib.load('vocab.bin')

In [5]:
fit = joblib.load('fit.bin')

In [6]:
test = Corpus('/Users/dclure/Projects/sent-order/data/test.json')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [7]:
test_x, test_y = zip(*test.xy(vocab))

HBox(children=(IntProgress(value=0, max=110615), HTML(value='')))

In [8]:
test_x = dv.transform(test_x)

In [9]:
def predict(corpus, dv, vocab, fit):
    """Predict order for all abstracts in a corpus.
    """
    gold_pred = []
    
    for ab in tqdm_notebook(corpus.abstracts):

        x, _ = zip(*ab.xy(vocab))
        x = dv.transform(x)

        pred = fit.predict(x).argsort().argsort()
        gold = list(range(len(pred)))

        gold_pred.append((gold, pred))
        
    return gold_pred

In [10]:
test_gp = predict(test, dv, vocab, fit)

HBox(children=(IntProgress(value=0, max=110615), HTML(value='')))

In [11]:
test_metrics = Metrics(test_gp)

In [12]:
test_metrics.avg_kt

/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


0.5497242452182497

In [13]:
test_metrics.avg_kt_by_sent_count

{2: 0.7726604605585498,
 3: 0.65081179967987923,
 4: 0.58244441167311756,
 5: 0.53318777292577746,
 6: 0.49394651289716585,
 7: 0.46670608718507617,
 8: 0.4414221019820026,
 9: 0.4230231999825404,
 10: 0.40853897375636328,
 11: 0.38036095640886081,
 12: 0.36902462866270913,
 13: 0.34831249765364009,
 14: 0.34473909324208729,
 15: 0.31047619047619096,
 16: 0.29380341880341898,
 17: 0.29411764705882354,
 18: 0.25000000000000006,
 19: 0.22222222222222221,
 20: 0.47719298245614034,
 21: 0.18412698412698414,
 24: 0.23550724637681159,
 30: -0.11264367816091954}